### Multiple Documents

In [183]:
import PyPDF2 as pdf
from PyPDF2 import PdfReader
import os
import pandas as pd
import re
import openai
import time
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
api_key_file = open("OPENAI_API_KEY", "r")
openai.api_key = api_key_file.read()

In [5]:
pdf_list = os.listdir("pdfs")

#### Assign ID

In [309]:
id_dict = dict(list(zip(pdf_list,range(0,len(pdf_list)))))
pdf_dict = {v: k for k, v in id_dict.items()}
pdf_dict

{0: 'Wi-Fi-6-OFDMA-App-Notes-091319-web.pdf',
 1: 'STBC.pdf',
 2: 'How to change the country code for WiFi.pdf',
 3: 'SPF53_SPF61_programmer guide.pdf',
 4: 'usb3-frequency-interference-papers.pdf',
 5: 'FCC-Rules-and-Regulations.pdf',
 6: 'programming uci shrinked.pdf',
 7: 'nor_nand_flash_guide.pdf',
 8: 'RadioChannelDFS.pdf'}

#### Iterate through

In [21]:
# Create dataframe for each individual pdf
df_list = []
for i,pdf in enumerate(pdf_list):
    
    cur_pdf = []
    
    with open("pdfs/" + pdf,"rb") as file:
        reader = PdfReader(file)
        for j in range(0, len(reader.pages)):
            cur_string = reader.pages[j].extract_text()
            cur_pdf.append("|start of page {} ".format(j+1)+cur_string + " end of page {}|".format(j+1))

    cur_df = pd.DataFrame({"page":list(range(1,len(cur_pdf)+1)),"text":cur_pdf})
    df_list.append((pdf,cur_df))

    

    


incorrect startxref pointer(1)


In [28]:
# Create embeddings for each page in each pdf
from openai.embeddings_utils import get_embedding
def embed(x):
    return get_embedding(x, engine="text-embedding-ada-002")

for i in df_list:
    i[1]["pg_embedding"] = i[1]["text"].apply(lambda x: embed(x))

In [45]:
# Save all embeddings
for df_pair in df_list:
    print(df_pair[0])
    np.save("Multiple Documents/embeddings/"+df_pair[0][0:-4]+"_ebded",np.array(list(df_pair[1]['pg_embedding'])))

Wi-Fi-6-OFDMA-App-Notes-091319-web.pdf
STBC.pdf
How to change the country code for WiFi.pdf
SPF53_SPF61_programmer guide.pdf
usb3-frequency-interference-papers.pdf
FCC-Rules-and-Regulations.pdf
programming uci shrinked.pdf
nor_nand_flash_guide.pdf
RadioChannelDFS.pdf


In [53]:
# Average Embeddings for Individual Documents
avg_embedding_pairs = []

for df_pair in df_list:
    avg_embedding = np.array(list(df_pair[1]["pg_embedding"])).mean(axis=0)
    avg_embedding_pairs.append((df_pair[0],avg_embedding))


In [67]:
avg_embedding_pairs_df = pd.DataFrame({"pdf":[i[0] for i in avg_embedding_pairs], "avg_embedding":[i[1] for i in avg_embedding_pairs], "id":list(range(0,len(avg_embedding_pairs)))})

In [74]:
avg_embedding_pairs_df.to_csv("Multiple Documents/avg_embedding_pairs_df")


In [85]:
entire_df = avg_embedding_pairs_df.copy()



for i,df_pair in enumerate(df_list):
    df_pair[1]["id"] = np.full(df_pair[1].shape[0],i)
    
    

In [100]:
final_df = pd.concat([i[1] for i in df_list]).set_index('id').join(entire_df.set_index('id'))


In [101]:
final_df.to_csv("Multiple Documents/final_pdf_df")

In [153]:
def get_suitable_documents(prompt, k_docs=3):
    embedded_prompt = embed(prompt)
    ranking = avg_embedding_pairs_df.copy()
    ranking["cos"] = ranking["avg_embedding"].apply(lambda x: x@embedded_prompt)
    ranked_df = ranking.sort_values(by="cos",ascending=False)
    print("looking at:",list(ranked_df["pdf"])[0:k_docs])
    return list(ranked_df["id"][0:k_docs])

In [317]:
def get_page_text(pdf_ID,pg):
    try:
        text = final_df[final_df["page"] == pg].loc[pdf_ID]["text"]
        pdf_name = pdf_dict[pdf_ID]
        return "|||From the manual " + pdf_name + ": " + text
    except:
        return ""
    


In [332]:
def get_suitable_text(prompt,k_docs=3, k_pages=10):
    #Find suitable manuals
    embedded_prompt = embed(prompt)
    doc_ids = get_suitable_documents(prompt,k_docs)
    filtered_df = final_df[final_df.index.isin(doc_ids)]
    #Merge Documents
    filtered_df["pg_cos"] = filtered_df["pg_embedding"].apply(lambda x: np.array(x)@embedded_prompt)
    ranked_filtered_df = filtered_df.sort_values(by="pg_cos",ascending=False)
    #display(ranked_filtered_df.head())
    #Find suitable pages
    #rank_filtered_df["text"][0:k_pages]
    
    idx_pg_pairs = sorted(list(zip(ranked_filtered_df.index[0:k_pages],ranked_filtered_df['page'][0:k_pages])))
    idx_pg_dict = {}
    for key, value in idx_pg_pairs:
        idx_pg_dict.setdefault(key, []).append(value)
        
    for key,value in idx_pg_dict.items():
        page_list = value
        new_list = []
        for i,page in enumerate(page_list):
            if i == len(page_list)-1:
                new_list.append(page)
                new_list.append(page+1)
                break
            if (page_list[i+1] - page) >= 4:
                new_list.append(page-1)
                new_list.append(page)
                new_list.append(page+1)
            else:
                new_list.append(page-1)
                for pgno in list(range(page,page_list[i+1])):
                    new_list.append(pgno)
        idx_pg_dict[key] = list(set([i for i in new_list if i != 0]))
    for ID,pages in idx_pg_dict.items():
        print("pages for {}:".format(pdf_dict[ID]),pages)
    
    final_list = []
    for ID, pages in idx_pg_dict.items():
        for page in pages:
            final_list.append(get_page_text(ID,page))
    return " ".join(final_list)
    
    
    
    


In [351]:
premise = "You’re a technical support bot that has access to manuals. Also always state the page number. The manual information is here: "
messages=[
    {"role": "system", "content": premise}
      ]
response_list = []
def conversation(response):
    response_list.append(response)
# If there is no stored info, store the first 10 similar pages based on the user's prompts

    if len(messages) == 1:
        messages[0]["content"] += get_suitable_text(response,k_pages=4)

    #print(len(messages))
    
    content = "Question: " + response + " Notes: You CAN ONLY USE INFO FROM THE MANUAL TEXT I REPEAT.  You CAN ONLY USE INFO FROM THE MANUAL TEXT. If there is nothing available in the text, SAY I don't have the info. Also state the pages in which you got the info from. Remember the manuals can be from different manuals. Say 'I have looked at the manual' if you understand this along with your response"
    messages.append({"role": "user", "content": content})
    
    completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-16k-0613",
    messages=messages,
    )
    message = completion["choices"][0]["message"]
    
    
    
    chat_response = completion.choices[0].message.content
    messages.append({"role": "assistant", "content": chat_response})
    
    for i in messages:
        if i['role'] == "user":
            pass
            #print("User: ", i['content'])
            #print("\n")
        elif i['role'] == "assistant":
            print("ChatGPT: ", i['content'])
        print("\n")
    
    #print(messages[0]["content"])
    #return (response,messages[-1]["content"])
conversation("What is the benefit of SLC NAND")

looking at: ['nor_nand_flash_guide.pdf', 'STBC.pdf', 'Wi-Fi-6-OFDMA-App-Notes-091319-web.pdf']
pages for nor_nand_flash_guide.pdf: [1, 2, 3, 5, 6, 7, 8, 9]




ChatGPT:  Based on the information available in the manual, the benefits of SLC (single-level cell) NAND flash are as follows:

1. High performance and write endurance: SLC NAND has the highest performance and endurance among NAND flash technologies. It offers the fastest NAND throughput and up to 100,000 program/erase cycles.

2. Best-in-class reliability: SLC NAND provides the best reliability among NAND flash technologies. It is commonly used in mission-critical systems where high performance and best-in-class reliability are required.

3. Lowest cost per bit for ≤1GB densities: SLC NAND is the most expensive NAND flash option but provides the lowest cost per bit when compared to other NAND flash technologies for densities up to 1GB.

4. Compatibility with the ONFI-synchronous interface: SLC NAND is compatible with the ONFI (